In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM
import torch
from datasets import load_dataset
import time

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model_name = "EleutherAI/gpt-j-6B"
# model_name = "tiiuae/falcon-7b"
# model_name = "mosaicml/mpt-7b"
# model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "meta-llama/Meta-Llama-3-8B"
# model_name = "NousResearch/Nous-Hermes-llama-2-7b"
# model_name = "databricks/dolly-v2-12b"  # right on the memory edge
model_name = "bigscience/bloom-7b1"


model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

2025-06-02 16:36:35.526061: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-02 16:36:35.539317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748882195.555088 3725944 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748882195.559900 3725944 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748882195.571962 3725944 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
prompt = (
    "Explain how black holes form and what happens inside them. "
    "Be detailed and use analogies a student could understand."
)

In [6]:
# --------------------------
# Tokenization
# --------------------------
start = time.time()
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
print(f"[Tokenization] {time.time() - start:.3f} sec")

# --------------------------
# Generation
# --------------------------
start = time.time()
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=512,
        do_sample=False
    )
torch.cuda.synchronize()
print(f"[Generation] {time.time() - start:.3f} sec")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Tokenization] 0.009 sec
[Generation] 16.495 sec
